In [1]:
%pip install datasets transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00


In [ ]:
import sklearn
import pandas as pd
import numpy as np
import random
import torch

seed = 1410

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Getting the dataset.

In [ ]:
from datasets import load_dataset
sem_eval_2018_task_1 = load_dataset('sem_eval_2018_task_1', 'subtask5.english', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

In [ ]:
%pip install deep-translator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


In [ ]:
from collections import Counter
from deep_translator import GoogleTranslator

# Convert to DataFrame
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Select half of the DataFrame
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Define label columns
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
label_counts = {label: half_df[label].sum() for label in label_columns}
print("Current Label Counts:", label_counts)

# Function to perform back translation
def back_translate(text, src_lang='en', mid_lang='fr'):
    translated_text = GoogleTranslator(source=src_lang, target=mid_lang).translate(text)
    back_translated_text = GoogleTranslator(source=mid_lang, target=src_lang).translate(translated_text)
    return back_translated_text

# Augment tweets
augmented_data = []
processed_tweets = set()  # To track processed tweets

for index, row in half_df.iterrows():
    tweet = row['Tweet']
    original_labels = row[label_columns].to_dict()

    # Skip if this tweet has already been processed
    if index in processed_tweets:
        continue

    # Perform back translation
    new_tweet = back_translate(tweet)

    # Append augmented data
    augmented_data.append({'Tweet': new_tweet, **original_labels})

    # Update processed tweets set
    processed_tweets.add(index)

print(f"Number of tweets augmented: {len(augmented_data)}")

# Combine original and augmented datasets
augmented_df = pd.DataFrame(augmented_data)
combined_df = pd.concat([half_df, augmented_df], ignore_index=True)

# Verify label counts
new_label_counts = {label: combined_df[label].sum() for label in label_columns}
print("New Label Counts:", new_label_counts)

# Save combined dataset to CSV
combined_df.to_csv('augmented_data_seed3.csv', index=False)

Current Label Counts: {'anticipation': 476, 'optimism': 979, 'trust': 185, 'joy': 1218, 'love': 341, 'anger': 1283, 'disgust': 1309, 'pessimism': 399, 'sadness': 1013, 'fear': 638, 'surprise': 177}
Number of tweets augmented: 3419
New Label Counts: {'anticipation': 952, 'optimism': 1958, 'trust': 370, 'joy': 2436, 'love': 682, 'anger': 2566, 'disgust': 2618, 'pessimism': 798, 'sadness': 2026, 'fear': 1276, 'surprise': 354}


BERT + 1/2 gold standard data.

In [ ]:
# Load your dataset into a DataFrame
df = pd.DataFrame(sem_eval_2018_task_1['train'])

label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Select half of the DataFrame
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Verify label distribution in the original and selected halves (optional)
original_label_distribution = df[label_columns].sum() / len(df)
selected_label_distribution = half_df[label_columns].sum() / len(half_df)

print("Original Label Distribution:")
print(original_label_distribution)
print("\nSelected Half Label Distribution:")
print(selected_label_distribution)

# Now `df_half` contains half of the original DataFrame with a label distribution close to the original

Original Label Distribution:
anticipation    0.143024
optimism        0.290143
trust           0.052208
joy             0.362240
love            0.102369
anger           0.372039
disgust         0.380521
pessimism       0.116262
sadness         0.293653
fear            0.181632
surprise        0.052793
dtype: float64

Selected Half Label Distribution:
anticipation    0.139222
optimism        0.286341
trust           0.054109
joy             0.356245
love            0.099737
anger           0.375256
disgust         0.382860
pessimism       0.116701
sadness         0.296285
fear            0.186604
surprise        0.051770
dtype: float64


In [ ]:
print(len(half_df))

3419


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(half_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(half_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5218, Val Loss: 0.4509
Epoch 2, Train Loss: 0.4168, Val Loss: 0.3811
Epoch 3, Train Loss: 0.3634, Val Loss: 0.3507
Epoch 4, Train Loss: 0.3273, Val Loss: 0.3340
Epoch 5, Train Loss: 0.2985, Val Loss: 0.3243
Test Accuracy: 26.20
Test F1-macro: 42.06
Test F1-micro: 65.83
Confusion Matrix:
Label: anticipation
[[2834    0]
 [ 425    0]]

Label: optimism
[[1759  357]
 [ 268  875]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1604  213]
 [ 288 1154]]

Label: love
[[2722   21]
 [ 410  106]]

Label: anger
[[1980  178]
 [ 317  784]]

Label: disgust
[[1920  240]
 [ 330  769]]

Label: pessimism
[[2884    0]
 [ 374    1]]

Label: sadness
[[2123  176]
 [ 407  553]]

Label: fear
[[2746   28]
 [ 271  214]]

Label: surprise
[[3089    0]
 [ 170    0]]



BERT + complete gold standard data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4715, Val Loss: 0.3784
Epoch 2, Train Loss: 0.3583, Val Loss: 0.3324
Epoch 3, Train Loss: 0.3098, Val Loss: 0.3147
Epoch 4, Train Loss: 0.2804, Val Loss: 0.3100
Epoch 5, Train Loss: 0.2583, Val Loss: 0.3068
Test Accuracy: 28.57
Test F1-macro: 49.31
Test F1-micro: 68.71
Confusion Matrix:
Label: anticipation
[[2789   45]
 [ 386   39]]

Label: optimism
[[1821  295]
 [ 314  829]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1667  150]
 [ 304 1138]]

Label: love
[[2666   77]
 [ 302  214]]

Label: anger
[[1897  261]
 [ 234  867]]

Label: disgust
[[1865  295]
 [ 269  830]]

Label: pessimism
[[2837   47]
 [ 317   58]]

Label: sadness
[[2157  142]
 [ 417  543]]

Label: fear
[[2720   54]
 [ 170  315]]

Label: surprise
[[3088    1]
 [ 169    1]]



BERT + 1/2 gold standard data + 1/2 synthetic data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("augmented_data_seed3.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data (assuming df_val remains unchanged)
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set (assuming df_test remains unchanged)
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4714, Val Loss: 0.3846
Epoch 2, Train Loss: 0.3537, Val Loss: 0.3386
Epoch 3, Train Loss: 0.3009, Val Loss: 0.3265
Epoch 4, Train Loss: 0.2658, Val Loss: 0.3159
Epoch 5, Train Loss: 0.2393, Val Loss: 0.3215
Test Accuracy: 27.83
Test F1-macro: 49.38
Test F1-micro: 68.84
Confusion Matrix:
Label: anticipation
[[2806   28]
 [ 386   39]]

Label: optimism
[[1794  322]
 [ 302  841]]

Label: trust
[[3105    1]
 [ 152    1]]

Label: joy
[[1652  165]
 [ 278 1164]]

Label: love
[[2609  134]
 [ 244  272]]

Label: anger
[[1899  259]
 [ 227  874]]

Label: disgust
[[1823  337]
 [ 263  836]]

Label: pessimism
[[2808   76]
 [ 321   54]]

Label: sadness
[[2070  229]
 [ 355  605]]

Label: fear
[[2727   47]
 [ 202  283]]

Label: surprise
[[3089    0]
 [ 170    0]]



#RoBERTa

RoBERTa + half GSD

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(half_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(half_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.6236, Val Loss: 0.5011
Epoch 2, Train Loss: 0.4850, Val Loss: 0.4759
Epoch 3, Train Loss: 0.4718, Val Loss: 0.4687
Epoch 4, Train Loss: 0.4491, Val Loss: 0.3972
Epoch 5, Train Loss: 0.4109, Val Loss: 0.3696
Epoch 6, Train Loss: 0.3871, Val Loss: 0.3553
Epoch 7, Train Loss: 0.3701, Val Loss: 0.3477
Epoch 8, Train Loss: 0.3537, Val Loss: 0.3439
Epoch 9, Train Loss: 0.3438, Val Loss: 0.3335
Epoch 10, Train Loss: 0.3321, Val Loss: 0.3293
Test Accuracy: 27.06
Test F1-macro: 43.85
Test F1-micro: 66.70
Confusion Matrix:
Label: anticipation
[[2834    0]
 [ 425    0]]

Label: optimism
[[1761  355]
 [ 253  890]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1638  179]
 [ 312 1130]]

Label: love
[[2672   71]
 [ 313  203]]

Label: anger
[[1923  235]
 [ 266  835]]

Label: disgust
[[1870  290]
 [ 265  834]]

Label: pessimism
[[2878    6]
 [ 366    9]]

Label: sadness
[[2051  248]
 [ 410  550]]

Label: fear
[[2725   49]
 [ 282  203]]

Label: surprise
[[3089    0]
 [ 17

RoBERTa + original df

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df = pd.DataFrame(sem_eval_2018_task_1['train'])
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5615, Val Loss: 0.4745
Epoch 2, Train Loss: 0.4526, Val Loss: 0.3941
Epoch 3, Train Loss: 0.3928, Val Loss: 0.3523
Epoch 4, Train Loss: 0.3646, Val Loss: 0.3411
Epoch 5, Train Loss: 0.3443, Val Loss: 0.3289
Epoch 6, Train Loss: 0.3286, Val Loss: 0.3167
Epoch 7, Train Loss: 0.3165, Val Loss: 0.3134
Epoch 8, Train Loss: 0.3066, Val Loss: 0.3068
Epoch 9, Train Loss: 0.2987, Val Loss: 0.3032
Epoch 10, Train Loss: 0.2907, Val Loss: 0.3036
Test Accuracy: 29.24
Test F1-macro: 50.21
Test F1-micro: 70.26
Confusion Matrix:
Label: anticipation
[[2821   13]
 [ 414   11]]

Label: optimism
[[1769  347]
 [ 271  872]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1618  199]
 [ 256 1186]]

Label: love
[[2597  146]
 [ 218  298]]

Label: anger
[[1934  224]
 [ 231  870]]

Label: disgust
[[1874  286]
 [ 245  854]]

Label: pessimism
[[2826   58]
 [ 296   79]]

Label: sadness
[[2134  165]
 [ 363  597]]

Label: fear
[[2684   90]
 [ 163  322]]

Label: surprise
[[3089    0]
 [ 17

RoBERTa + half GSD + half synthetic data

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df = pd.read_csv("augmented_data_seed3.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5549, Val Loss: 0.4755
Epoch 2, Train Loss: 0.4492, Val Loss: 0.3881
Epoch 3, Train Loss: 0.3856, Val Loss: 0.3490
Epoch 4, Train Loss: 0.3557, Val Loss: 0.3362
Epoch 5, Train Loss: 0.3344, Val Loss: 0.3279
Epoch 6, Train Loss: 0.3167, Val Loss: 0.3183
Epoch 7, Train Loss: 0.3036, Val Loss: 0.3148
Epoch 8, Train Loss: 0.2929, Val Loss: 0.3166
Epoch 9, Train Loss: 0.2843, Val Loss: 0.3084
Epoch 10, Train Loss: 0.2734, Val Loss: 0.3097
Test Accuracy: 28.75
Test F1-macro: 50.97
Test F1-micro: 69.95
Confusion Matrix:
Label: anticipation
[[2797   37]
 [ 389   36]]

Label: optimism
[[1783  333]
 [ 255  888]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1647  170]
 [ 278 1164]]

Label: love
[[2609  134]
 [ 223  293]]

Label: anger
[[1931  227]
 [ 267  834]]

Label: disgust
[[1888  272]
 [ 282  817]]

Label: pessimism
[[2781  103]
 [ 289   86]]

Label: sadness
[[2126  173]
 [ 344  616]]

Label: fear
[[2664  110]
 [ 148  337]]

Label: surprise
[[3089    0]
 [ 17